In [ ]:
%pip install datasets
%pip install transformers
%pip install torch torchvision torchaudio
%pip install pydantic
%pip install transformers[torch]
%pip install accelerate -U
%pip install scikit-learn
%pip install evaluate
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable



### Parse the movie script, and save it in json format

In [1]:
import json
import sys
from  collections import Counter

sys.path.append('..')

from src.data.parse_movie_script import MovieScriptParser

# Parse the dataset in structured format and output a json compatible to parse with Hugging face datasets library.
# The output will be a json file with the following structure:
# { "text": ..., "label": }
parsed_object = MovieScriptParser.from_text_file("../data/raw/5thelement.txt")

# Print the stats for the parsed script
print("Script basic stats:")
print(json.dumps(parsed_object.stats, indent=2))

# Print counts per class
class_map = parsed_object.get_vocabulary_to_label_mapping()
class_counter = Counter()
for scene in parsed_object.scenes:
    for entry in scene.entries:
        if entry.dialogue is not None:
            class_counter[entry.dialogue.character] += 1

print("Class counts (character speeches):")
for key, value in class_counter.most_common():
    print(f"{key} with class id {class_map[key]}: {class_counter[key]} ({class_counter[key] / parsed_object.stats['total_dialogues']})")

# Save the parsed script in json format
parsed_object.save_character_dialogue_dataset_in_json_format("../data/parsed/5thelement.json")

# Print a few lines of the saved file to see the format
with open("../data/parsed/5thelement.json", "r") as f:
    for i in range(5):
        print(f.readline().strip())

Script basic stats:
{
  "total_words": 21849,
  "total_dialogues": 943,
  "total_words_in_dialogues": 9457,
  "total_characters": 83,
  "total_scenes": 281
}
Class counts (character speeches):
KORBEN with class id 41: 244 (0.25874867444326616)
CORNELIUS with class id 21: 99 (0.10498409331919406)
ZORG with class id 82: 68 (0.07211028632025451)
LOC RHOD with class id 44: 52 (0.05514316012725345)
PRESIDENT with class id 59: 48 (0.05090137857900318)
LEELOO with class id 43: 48 (0.05090137857900318)
MUNRO with class id 55: 36 (0.03817603393425239)
PROFESSOR with class id 65: 24 (0.02545068928950159)
MACTILBURGH with class id 45: 16 (0.016967126193001062)
STAEDERT with class id 72: 15 (0.015906680805938492)
CAPTAIN with class id 8: 15 (0.015906680805938492)
RIGHT ARM with class id 66: 15 (0.015906680805938492)
PRIEST with class id 61: 14 (0.014846235418875928)
FINGER (V.O.) with class id 29: 13 (0.013785790031813362)
DAVID with class id 22: 12 (0.012725344644750796)
VOICE (O.S.) with class i

### Create a basic traind and test split of the dataset

In [ ]:
from datasets import load_dataset

def split_dataset_from_json(json_file: str, test_size: float = 0.2):
    """
    Load a dataset from a json file and split it into train and test sets by using datasets library from Hugging Face.

    Args:
        json_file (str): The path to the json file containing the dataset.
        test_size (float): The proportion of the dataset to include in the test split.
    """
    # Load dataset from local json file 
    dataset = load_dataset("json", data_files=json_file)

    # Split the dataset into a simple train, validation and test sets
    train_test_split = dataset['train'].train_test_split(test_size=test_size)
    train_dataset = train_test_split['train']
    test_dataset = train_test_split['test']

    print(f"Train Size: {len(train_dataset)}, Test Size: {len(test_dataset)}")

    return train_dataset, test_dataset

train_dataset, test_dataset = split_dataset_from_json("../data/parsed/5thelement.json")

/Users/mcelikik/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mcelikik/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 943 examples [00:00, 242652.07 examples/s]

Train Size: 754, Test Size: 189


### Train a first exploratory Hugging face Transformer model with typical params and do basic evaluation

In [3]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import evaluate

### Train model
model_name = "microsoft/deberta-base"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Add a padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use <EOS> as <PAD>

# Tokenize the data
def preprocess_function(examples):
    tokenized = tokenizer(
        examples["text"],  # raw text to be tokenized
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt",
    )
    tokenized["labels"] = examples["label"]  # add the labels to the tokenized input
    return tokenized

encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Sanity check: print a few examples of the encoded dataset
print("Encoded Train Dataset:")
print(encoded_train_dataset["input_ids"][:2])
print(encoded_train_dataset["attention_mask"][:2])
print(encoded_train_dataset["labels"][:2])

# Load the model
num_labels = len(parsed_object.character_vocabulary)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)

# Add a padding token if not present
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    save_strategy="epoch",
    logging_steps=30,
    warmup_steps=10,  # learning rate will be gradually increased during the first 10 steps
    load_best_model_at_end=True,
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    metric_acc = evaluate.load("accuracy")
    metric_f1 = evaluate.load("f1")

    accuracy = metric_acc.compute(predictions=predictions, references=labels)
    f1 = metric_f1.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"],
    }


# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

Map: 100%|██████████| 189/189 [00:00<00:00, 4898.43 examples/s]


Encoded Train Dataset:
[[1, 734, 4783, 9069, 328, 1437, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 463, 5, 6675, 9, 41, 20285, 116, 1437, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                               
 50%|█████     | 24/48 [00:27<00:19,  1.24it/s]

{'eval_loss': 4.040040493011475, 'eval_accuracy': 0.23809523809523808, 'eval_f1': 0.008547008547008548, 'eval_runtime': 4.9674, 'eval_samples_per_second': 38.048, 'eval_steps_per_second': 4.832, 'epoch': 1.0}


 62%|██████▎   | 30/48 [00:41<00:31,  1.76s/it]

...

                                               
100%|██████████| 48/48 [01:26<00:00,  2.16s/it]

{'eval_loss': 3.765052080154419, 'eval_accuracy': 0.23809523809523808, 'eval_f1': 0.008547008547008548, 'eval_runtime': 5.9513, 'eval_samples_per_second': 31.758, 'eval_steps_per_second': 4.033, 'epoch': 2.0}


100%|██████████| 48/48 [01:31<00:00,  1.91s/it]

{'train_runtime': 91.8754, 'train_samples_per_second': 16.414, 'train_steps_per_second': 0.522, 'train_loss': 4.055851300557454, 'epoch': 2.0}


TrainOutput(global_step=48, training_loss=4.055851300557454, metrics={'train_runtime': 91.8754, 'train_samples_per_second': 16.414, 'train_steps_per_second': 0.522, 'total_flos': 115659434542080.0, 'train_loss': 4.055851300557454, 'epoch': 2.0})

### Compute confusion matrix and additional metrics per class

In [4]:
# Compute predictions on test set to compute additional metrics
def get_predictions_and_labels(test_dataset, trainer):
    import numpy as np

    # Predict on the evaluation dataset
    predictions = trainer.predict(test_dataset)

    # Extract predictions and true labels
    y_pred = np.argmax(predictions.predictions, axis=1)
    y_true = predictions.label_ids
    return y_pred, y_true


def compute_precision_recall_per_class(y_pred, y_true, num_labels):

    from sklearn.metrics import confusion_matrix
    import numpy as np

    def compute_precision_recall(confusion_matrix):
        """
        Compute precision and recall for each class from a confusion matrix.
        
        :param confusion_matrix: 2D array, where rows are actual classes
                                and columns are predicted classes.
        :return: Dictionary with precision and recall for each class.
        """
        num_classes = confusion_matrix.shape[0]
        metrics = []
        
        for i in range(num_classes):
            # True Positives (diagonal element)
            TP = confusion_matrix[i, i]
            
            # False Positives (sum of column i, excluding TP)
            FP = np.sum(confusion_matrix[:, i]) - TP
            
            # False Negatives (sum of row i, excluding TP)
            FN = np.sum(confusion_matrix[i, :]) - TP
            
            # Precision and Recall
            precision = float(TP / (TP + FP)) if (TP + FP) > 0 else 0
            recall = float(TP / (TP + FN)) if (TP + FN) > 0 else 0
            
            metrics.append({'Class': i, 'Precision': precision, 'Recall': recall})
        
        return metrics

    print("# predictions in test set: {}".format(len(y_pred)))
    print("Predictions on test set:")
    print(y_pred)
    print("True labels:")
    print(y_true)

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=range(num_labels))

    # Compute precision and recall per class
    metrics_per_class = compute_precision_recall(cm)
    metrics_per_class = sorted(metrics_per_class, key=lambda x: x['Recall'], reverse=True)
    
    print("Precision and recall per class:")
    print(metrics_per_class)

y_pred, y_true = get_predictions_and_labels(encoded_test_dataset, trainer)
compute_precision_recall_per_class(y_pred, y_true, num_labels)

100%|██████████| 24/24 [00:04<00:00,  5.67it/s]

# predictions in test set: 189
Predictions on test set:
[41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41]
True labels:
[41 66 69 31  8 55 82 80 61 82 22 69 82 82 41 55 37 43 43 59 59 41 59 59
 41 41 59 21 41 41 23 45  8 41 41 41 44 55 53 44 59 21 66 53 61 41 21 41
 59 22 55 82 24 41 43 41 59 21 10 15 55 57 43  8  1 41 41 65 82 82 41 41
 41 43 43 21 59 41 82 41 68 65 41 13 57 73 82 41 21 43 41 21 24 41 53  1
 45 47 21 29 21 41 41 35 21 77 41 55 41 60 41  1 24 82 61 79 54

### Train and evaluate on oversampled dataset (oversample each class proportionally to max class size)

In [6]:
import random
from datasets import Dataset

def oversample_dataset(dataset: Dataset, class_count_threshold=0):
    """
    Oversamples the dataset to balance the class distribution. Ignore classes with count less or equal to class_count_threshold.
    Args:
        dataset (Dataset): A dataset object containing examples with a "label" field.
        class_count_threshold (int): The minimum count of a class to be considered for oversampling.
    Returns:
        Dataset: A new dataset object with balanced class distribution by oversampling the minority classes.
    """
    class_counts = Counter(dataset["label"])

    max_count = max(class_counts.values())
    examples_by_class = {label: [] for label in class_counts}
    
    for example in dataset:
        examples_by_class[example["label"]].append(example)
    
    balanced_examples = []
    for _, examples in examples_by_class.items():
        if len(examples) > class_count_threshold:
            balanced_examples.extend(random.choices(examples, k=max_count))
        else:
            balanced_examples.extend(examples)
    
    random.shuffle(balanced_examples)
    return Dataset.from_list(balanced_examples)

train_dataset_oversampled = oversample_dataset(train_dataset, class_count_threshold=2)

print("Training dataset size:", len(train_dataset))
print("Oversampled training dataset size:", len(train_dataset_oversampled))

encoded_train_dataset_oversampled = train_dataset_oversampled.map(preprocess_function, batched=True)

# Reset model weights to previous state
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset_oversampled,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluate the model
y_pred, y_true = get_predictions_and_labels(encoded_test_dataset, trainer)
compute_precision_recall_per_class(y_pred, y_true, num_labels)

Training dataset size: 754
Oversampled training dataset size: 7618


Map: 100%|██████████| 7618/7618 [00:00<00:00, 13454.65 examples/s]
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  6%|▋         | 30/478 [00:27<07:28,  1.00s/it]

{'loss': 4.3595, 'grad_norm': 3.3645477294921875, 'learning_rate': 9.572649572649575e-06, 'epoch': 0.13}


 13%|█▎        | 60/478 [00:56<06:18,  1.10it/s]

{'loss': 4.122, 'grad_norm': 3.8560879230499268, 'learning_rate': 8.931623931623933e-06, 'epoch': 0.25}


...

{'eval_loss': 3.7538673877716064, 'eval_accuracy': 0.07936507936507936, 'eval_f1': 0.06009914529914529, 'eval_runtime': 8.2946, 'eval_samples_per_second': 22.786, 'eval_steps_per_second': 2.893, 'epoch': 2.0}


100%|██████████| 478/478 [08:16<00:00,  1.04s/it]


{'train_runtime': 496.0503, 'train_samples_per_second': 30.715, 'train_steps_per_second': 0.964, 'train_loss': 2.958762731512221, 'epoch': 2.0}


100%|██████████| 24/24 [00:04<00:00,  5.50it/s]

# predictions in test set: 189
Predictions on test set:
[54 10 17 76 44 53 10 80 24 54 24 57 20 40 56 12 10 24 24 54  1 24 10 10
 53 53  1 45 24 54 54 17 17 15 54 10 53 53 53 53 44 34 10 53 24 24 40 10
 10 10 10 54 24 34 34 24 45 40 79 76 24 34 53 76 12 31 17 65 17 45 34 10
 56 57 34 10 45 12 72 53 68 65 53 15 57 24  2 24 40 31 53 24 24 10 53 40
 68 76 34 79 31 53 31 31 53 65 45 10 43  1 56 53 24 53 65 20 53 40 29 31
 10 10 10 45 79 80 65 45 34 40 71 45 54 56 45 40 54 53 10 57 10 54 76 24
 34 31 10 76 76 34 29 71 24 53 57 53 71 10 10  1 10 53  1 71  2 10 45 76
 10 24 29 24 40 10 57  1 71 68 68 34 22 53 12 24 10 65 53 24 76]
True labels:
[41 66 69 31  8 55 82 80 61 82 22 69 82 82 41 55 37 43 43 59 59 41 59 59
 41 41 59 21 41 41 23 45  8 41 41 41 44 55 53 44 59 21 66 53 61 41 21 41
 59 22 55 82 24 41 43 41 59 21 10 15 55 57 43  8  1 41 41 65 82 82 41 41
 41 43 43 21 59 41 82 41 68 65 41 13 57 73 82 41 21 43 41 21 24 41 53  1
 45 47 21 29 21 41 41 35 21 77 41 55 41 60 41  1 24 82 61 79 54

### Use weighted cross-entropy loss to simulate oversampling with less computational effort

In [52]:
import torch
import torch.nn as nn
import numpy as np

# Reset model weights to previous state
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)

def compute_class_weights(dataset: Dataset, num_labels: int, min_count=0, top_k=None):
    """
    Compute class weights for
    Args:
        dataset (Dataset): A dataset object containing examples with a "label" field.
        num_labels (int): The number of classes in the dataset.
        min_count (int): The minimum count of a class to be considered for computing class weights.
    Returns:
        dict: A dictionary with class weights for each class compared to the class with maximum count.
    """
    class_counts = Counter(dataset["label"])

    if top_k is not None:
        class_counts = dict(class_counts.most_common(top_k))

    max_count = max(class_counts.values())
    class_weights = {label: max_count / count for label, count in class_counts.items()}
    class_weights_list = [1.0] * (num_labels)

    for label, weight in class_weights.items():
        if class_counts[label] > min_count:
            class_weights_list[label] = weight
            
    return class_weights_list

# Compute class weights
class_weights = compute_class_weights(train_dataset, num_labels=num_labels, top_k=10)
for i, weight in enumerate(class_weights):
    if weight > 1.0:
        print(f"Class {i} weight: {weight}")
class_weights = torch.tensor(class_weights, dtype=torch.float32)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
class_weights = class_weights.to(device)

# Define the loss function with class weights
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Define custom compute_loss_func
def compute_loss_func(outputs, labels, num_items_in_batch=None):
    logits = outputs.logits
    loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
    return loss

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    compute_metrics=compute_metrics,
    compute_loss_func=compute_loss_func,
)

training_args.num_train_epochs = 10

trainer.train()

# Evaluate the model
y_pred, y_true = get_predictions_and_labels(encoded_test_dataset, trainer)
compute_precision_recall_per_class(y_pred, y_true, num_labels)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Class 21 weight: 2.513157894736842
Class 43 weight: 4.5476190476190474
Class 44 weight: 4.2444444444444445
Class 45 weight: 14.692307692307692
Class 55 weight: 6.586206896551724
Class 59 weight: 5.96875
Class 65 weight: 9.095238095238095
Class 66 weight: 15.916666666666666
Class 82 weight: 3.3508771929824563














                                      
                                             

  0%|          | 0/48 [15:39<?, ?it/s]         



{'eval_loss': 4.153419017791748, 'eval_accuracy': 0.06878306878306878, 'eval_f1': 0.0058285248003730255, 'eval_runtime': 4.1092, 'eval_samples_per_second': 45.995, 'eval_steps_per_second': 5.841, 'epoch': 1.0}


                                      
  0%|          | 0/48 [15:52<?, ?it/s]          

...

{'eval_loss': 3.0337133407592773, 'eval_accuracy': 0.1111111111111111, 'eval_f1': 0.0308969131510387, 'eval_runtime': 5.3574, 'eval_samples_per_second': 35.278, 'eval_steps_per_second': 4.48, 'epoch': 10.0}


                                      
100%|██████████| 240/240 [07:11<00:00,  1.80s/it]


{'train_runtime': 431.9662, 'train_samples_per_second': 17.455, 'train_steps_per_second': 0.556, 'train_loss': 3.3202081044514973, 'epoch': 10.0}


100%|██████████| 24/24 [00:04<00:00,  5.32it/s]

# predictions in test set: 189
Predictions on test set:
[44 43 59 59 41 65 44 59 41 44 44 59 44 65 44 59 44 41 44 45 44 44 44 59
 45 59 59 21 59 59 59 59 82 59 59 45 44 21 59 59 59 21 21 59 44 45 45 59
 59 45 44 44 59 44 59 44 59 65 59 44 45 44 59 44 59 21 21 59 43 59 59 21
 59 59 59 44 59 59 21 44 43 44 45 59 44 44 59 44 43 45 59 44 59 21 44 82
 21 82 59 21 59 41 59 44 59 82 41 59 45 82 82 44 82 44 44 59 21 59 44 44
 59 82 82 21 44 82 59 59 45 43 43 82 59 59 41 44 44 59 59 59 44 59 21 65
 59 44 44 44 44 21 59 59 21 59 59 59 59 44 43 59 21 41 59 59 82 59 59 59
 44 41 59 21 82 21 59 21 59 44 44 59 59 44 41 59 59 59 45 59 59]
True labels:
[41 34 22 45  0 21 41 80 56 66 82 41 41 65 76 41 41 82 82 59 44 55 41 55
 21  1 41 73 66 71 59 61 82 21 72 68 41 59 55 64 21 66 21 43 41 59 24 76
 41 55 41 41 41 41  7 77 41 21 59 41 21 41 41 44  4 43 43 82 59 43 21 82
 80 55 68 82 41 79 21 41 43 21 82 59 41 73 44 41 41 40 41 29 61 21 41 80
 33 39 41 56 55 21 72 29 43 66 65 41 68 41 21 24 41 53 82 59 72

### Hyperparameter tunning

In [ ]:
! pip install optuna
! pip install ray[tune]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.1 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
zsh:1: no matches found: ray[tune]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:

### Split the dataset into train, validation and test sets

from datasets import load_dataset

# Load dataset from local json file 
dataset = load_dataset("json", data_files="./data/parsed/5thelement.json")

# Split the dataset into a simple train, validation and test sets
train_test_split = dataset['train'].train_test_split(test_size=0.4)

train_dataset = train_test_split['train']

temp_split_dataset = train_test_split['test'].train_test_split(test_size=0.5)
val_dataset = temp_split_dataset['train']
test_dataset = temp_split_dataset['test']

print(f"Train Size: {len(train_dataset)}, Test Size: {len(test_dataset)}, Val Size: {len(val_dataset)}")

Train Size: 565, Test Size: 189, Val Size: 189


In [58]:
# Run hyperparameter search

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

train_dataset_oversampled = oversample_dataset(train_dataset, class_count_threshold=2)

encoded_train_dataset = train_dataset_oversampled.map(preprocess_function, batched=True)
encoded_test_dataset = test_dataset.map(preprocess_function, batched=True)
encoded_val_dataset = val_dataset.map(preprocess_function, batched=True)

trainer_hp = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    compute_metrics=compute_metrics,
)

best_run = trainer_hp.hyperparameter_search(n_trials=2, direction="maximize")

print(best_run)

Map: 100%|██████████| 189/189 [00:00<00:00, 8285.85 examples/s]
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-11-30 21:54:22,213] A new study created in memory with name: no-name-5662d24d-8aac-454c-971a-0841b2ea97d4
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                 
 25%|██▌       | 189/756 [07:07<02:22,  3.99it/s]

{'loss': 4.4345, 'grad_norm': 4.090267658233643, 'learning_rate': 8.854597805338979e-06, 'epoch': 0.21}


                                                 
 25%|██▌       | 189/756 [07:18<02:22,  3.99it/s]

{'loss': 4.3045, 'grad_norm': 9.708539009094238, 'learning_rate': 6.482830536051752e-06, 'epoch': 0.42}


                                                 
 25%|██▌       | 189/756 [07:25<02:22,  3.99it/s]

...

{'loss': 3.4228, 'grad_norm': 7.428894519805908, 'learning_rate': 1.6905178747060257e-06, 'epoch': 1.69}















                                                 

                                         
 25%|██▌       | 189/756 [09:06<02:22,  3.99it/s]



{'eval_loss': 3.4981696605682373, 'eval_accuracy': 0.2222222222222222, 'eval_f1': 0.008658008658008658, 'eval_runtime': 5.6005, 'eval_samples_per_second': 33.747, 'eval_steps_per_second': 4.285, 'epoch': 2.0}


                                                 
100%|██████████| 142/142 [01:12<00:00,  1.95it/s]
[I 2024-11-30 21:56:35,202] Trial 1 finished with value: 0.23088023088023088 and parameters: {'learning_rate': 1.0143107248236155e-05, 'num_train_epochs': 2, 'seed': 19, 'per_device_train_batch_size': 8}. Best is trial 0 with value: 0.23088023088023088.


{'train_runtime': 72.7923, 'train_samples_per_second': 15.524, 'train_steps_per_second': 1.951, 'train_loss': 3.7333725405410982, 'epoch': 2.0}
BestRun(run_id='0', objective=0.23088023088023088, hyperparameters={'learning_rate': 1.0435775984863796e-05, 'num_train_epochs': 1, 'seed': 7, 'per_device_train_batch_size': 4}, run_summary=None)


In [ ]:
# Train and evaluate the model with the best hyperparameters
for n, v in best_run.hyperparameters.items():
    setattr(trainer_hp.args, n, v)

trainer_hp.train()

# Evaluate the model
### Evaluate the model
results = trainer_hp.evaluate(eval_dataset=encoded_test_dataset)
print("Evaluation Results on Test Set:", results)

y_pred, y_true = get_predictions_and_labels(encoded_test_dataset, trainer)

compute_precision_recall_per_class(y_pred, y_true, num_labels)

                                                 
 25%|██▌       | 189/756 [09:27<02:22,  3.99it/s]

{'loss': 4.0689, 'grad_norm': 11.085065841674805, 'learning_rate': 8.88268156424581e-06, 'epoch': 0.16}


                                                 
 25%|██▌       | 189/756 [09:36<02:22,  3.99it/s]

{'loss': 3.7027, 'grad_norm': 10.091877937316895, 'learning_rate': 7.206703910614526e-06, 'epoch': 0.32}


                                                 
 25%|██▌       | 189/756 [09:46<02:22,  3.99it/s]

...

{'train_runtime': 88.6759, 'train_samples_per_second': 8.503, 'train_steps_per_second': 2.131, 'train_loss': 3.6040518523524048, 'epoch': 1.0}


100%|██████████| 24/24 [00:06<00:00,  3.99it/s]


Evaluation Results on Test Set: {'eval_loss': 3.1965441703796387, 'eval_accuracy': 0.328042328042328, 'eval_f1': 0.01073965009527109, 'eval_runtime': 6.1758, 'eval_samples_per_second': 30.603, 'eval_steps_per_second': 3.886, 'epoch': 1.0}


100%|██████████| 24/24 [00:05<00:00,  4.46it/s]

# predictions in test set: 189
Predictions on test set:
[41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41
 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41 41]
True labels:
[66 21 43 41 41 59 43 45 54 41 44 61 21 41 74 82 41 41 34 21  7 21 82 61
 41 41 41 59 55 41 21 26 82 41 41 77  8 41 48 41 41 78 41 66 44 41 55 80
 44 41  3 21 41 21 41 41 69 41 41 20 44 41 41 21 72 21 41 44 45 41 21 44
 55 41 72 41  1 44 44 82 66 43 41 31 44 41 22 55 41 68 22 72 41 41 21 82
 41 43 77 55 59 50 34 29 41 61 68 21 12 43 10 17 41 41 82 41 21